<a href="https://colab.research.google.com/github/ghostfm3/ksks/blob/master/word2vec0test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving 789_ruby_5639.zip to 789_ruby_5639.zip


In [3]:
from google.colab import files
uploaded = files.upload()

Saving wagahaiwa_nekodearu.txt to wagahaiwa_nekodearu.txt


In [4]:
with open('wagahaiwa_nekodearu.txt', mode='rt', encoding='cp932') as rf:
    for i, line in enumerate(rf):
        if i < 5: # この数字を変えることで表示する行数が変化
            print(line.strip())

吾輩は猫である
夏目漱石

-------------------------------------------------------
【テキスト中に現れる記号について】


In [5]:
with open('wagahaiwa_nekodearu.txt', mode='rt', encoding='cp932') as rf:
    with open('吾輩は猫である.txt', mode='wt', encoding='cp932') as wf:
        for i, line in enumerate(rf):
            if i > 23 and 2358 >= i:
                wf.write(line)

In [6]:
import re
pattern1 = re.compile(r'《.*?》', re.MULTILINE)
pattern2 = re.compile(r'｜', re.MULTILINE)
pattern3 = re.compile(r'［＃.*?］', re.MULTILINE)
with open('吾輩は猫である.txt', mode='rt', encoding='cp932') as rf:
    with open('吾輩は猫である_ルビなし.txt', mode='wt', encoding='cp932') as wf:
        for i, line in enumerate(rf):
            line = pattern1.sub('',line)
            line = pattern2.sub('',line)
            line = pattern3.sub('',line)
            wf.write(line)

In [7]:
files.download('吾輩は猫である_ルビなし.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [8]:
!apt-get -q -y install sudo file mecab libmecab-dev mecab-ipadic-utf8 git curl python-mecab > /dev/null
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git > /dev/null 
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n > /dev/null 2>&1
!pip install mecab-python3 > /dev/null
!echo mecab-config --dicdir"/mecab-ipadic-neologd"

Cloning into 'mecab-ipadic-neologd'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 75 (delta 5), reused 54 (delta 0), pack-reused 0
Unpacking objects: 100% (75/75), done.
mecab-config --dicdir/mecab-ipadic-neologd


In [15]:
pip install unidic-lite

     |████████████████████████████████| 47.3MB 97kB/s 
  Created wheel for unidic-lite: filename=unidic_lite-1.0.7-cp36-none-any.whl size=47556594 sha256=5c9ef9fdcd17624b264e53d3f9f5fe6c5834d0aafbb24885fd23e3bad9fe90c2
  Stored in directory: /root/.cache/pip/wheels/a8/82/7d/086724645e33a575aafd0b1dae2835c37d2c00c6a0a96ee3a0
Successfully built unidic-lite


In [16]:
import MeCab
m = MeCab.Tagger('-Owakati')
result = m.parse('形態素解析を練習で実行してみる')
print(result)

形態 素 解析 を 練習 で 実行 し て みる 



In [17]:
import MeCab
path = '-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd'
m = MeCab.Tagger(path)
result = m.parse('形態素解析を実行してみる')
print(result)

形態素解析	名詞,固有名詞,一般,*,*,*,形態素解析,ケイタイソカイセキ,ケイタイソカイセキ
を	助詞,格助詞,一般,*,*,*,を,ヲ,ヲ
実行	名詞,サ変接続,*,*,*,*,実行,ジッコウ,ジッコー
し	動詞,自立,*,*,サ変・スル,連用形,する,シ,シ
て	助詞,接続助詞,*,*,*,*,て,テ,テ
みる	動詞,非自立,*,*,一段,基本形,みる,ミル,ミル
EOS



In [18]:
import MeCab
def mecab_tokenizer(text):
    m = MeCab.Tagger("-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd")
    node = m.parseToNode(text)
    wordtype_list = ['名詞','動詞','形容詞']
    subtype_list = ["数", "非自立", "代名詞","接尾"]
    output = []
    while node:
        if node.surface != "":
            wordtype = node.feature.split(",")[0]
            subtype = node.feature.split(",")[1]
            original = node.feature.split(",")[6]
            if wordtype in wordtype_list and subtype not in subtype_list and original != '*':
                output.append(original)
        node = node.next
        if node is None:
            break
    return output

In [19]:
text = 'これで望み通りの形態素解析を行う関数定義ができます．'
mecab_tokenizer(text)

['望み', '形態素解析', '行う', '関数', '定義', 'できる']

In [20]:
import pandas as pd
text_data = pd.read_table('吾輩は猫である_ルビなし.txt', encoding='cp932', header = None)
text_data.columns = ['text']
text_data['wakati'] = text_data['text'].apply(lambda x: ' '.join(mecab_tokenizer(x)))

In [21]:
text_data.wakati.to_csv('wakati.csv', sep='\n', header=True, index=False)

In [22]:
from gensim.models import word2vec
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
sentence_data = word2vec.LineSentence('wakati.csv')
model_w2v = word2vec.Word2Vec(sentence_data,
                         sg=1,        # Skip-gram
                         size=200,    # 次元数
                         min_count=5, # min_count回未満の単語を破棄
                         window=5,    # 文脈の最大単語数
                         hs=1,        # 階層ソフトマックス(ネガティブサンプリングするなら0)
                         negative=5,  # ネガティブサンプリング
                         iter=50      # Epoch数
                         )

2020-12-05 06:52:25,184 : INFO : collecting all words and their counts
2020-12-05 06:52:25,185 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-12-05 06:52:25,216 : INFO : collected 11128 word types from a corpus of 67030 raw words and 2256 sentences
2020-12-05 06:52:25,222 : INFO : Loading a fresh vocabulary
2020-12-05 06:52:25,231 : INFO : effective_min_count=5 retains 2476 unique words (22% of original 11128, drops 8652)
2020-12-05 06:52:25,234 : INFO : effective_min_count=5 leaves 52863 word corpus (78% of original 67030, drops 14167)
2020-12-05 06:52:25,247 : INFO : deleting the raw counts dictionary of 11128 items
2020-12-05 06:52:25,250 : INFO : sample=0.001 downsamples 36 most-common words
2020-12-05 06:52:25,251 : INFO : downsampling leaves estimated 43537 word corpus (82.4% of prior 52863)
2020-12-05 06:52:25,256 : INFO : constructing a huffman tree from 2476 words
2020-12-05 06:52:25,309 : INFO : built huffman tree with maximum node depth 13
20

In [33]:
s = input('類似度の計測をしたい単語を入力：')
try:
    results = model_w2v.most_similar(positive=[s], topn=20)
    for i, result in enumerate(results):
        print(i+1, '\t', '{0:.5f}'.format(result[1]), ' ', result[0])
except KeyError:
    print('{}が辞書の中に入っていない'.format(s))

類似度の計測をしたい単語を入力：人間
1 	 0.36756   動物
2 	 0.35593   歴史
3 	 0.32199   する
4 	 0.31371   証明
5 	 0.31321   衣服
6 	 0.29793   案出
7 	 0.29272   古物
8 	 0.29130   糞
9 	 0.29111   縛する
10 	 0.29008   無能
11 	 0.28598   能力
12 	 0.28529   珍重
13 	 0.28410   もっとも
14 	 0.28299   愛
15 	 0.28049   個人
16 	 0.27950   赤裸
17 	 0.27722   同一
18 	 0.27673   なる
19 	 0.27572   出来る
20 	 0.27512   結果


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
